# SCENARI D'USO

In [1]:
import carla, time, sys, threading, math, random, paho.mqtt.client as mqtt
from agents.navigation.basic_agent import BasicAgent
from enum import Enum
from Adas import Forward_collision_warning_mqtt

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

# FUNZIONI DI SUPPORTO

## Gestione simulazione

In [3]:
# Simulazione sensore umidità dell'asflato
def get_asphalt_friction_coefficient():
    world = client.get_world()
    weather = world.get_weather()
    weather.precipitation_deposits
    weather.wetness
    return 1

In [4]:
def stop_simulation():
    global run
    run = False
    print("stop")

In [5]:
def set_synchronous_mode(synchronous_mode):
    settings = world.get_settings()
    settings.synchronous_mode = synchronous_mode
    world.apply_settings(settings)
    world.tick()

In [6]:
def simulation_loop():
    print("run")
    set_synchronous_mode(True)
    while run:
        time.sleep(0.05)
        world.tick()
    print("stop")
    set_synchronous_mode(False)


In [7]:
def run_sync_simulation():
    threading.Thread(target=simulation_loop).start()

## ADAS

In [8]:
def attach_adas(world, vehicle, min_ttc = 0.5):
    adas = Forward_collision_warning_mqtt(
        world = world,
        attached_vehicle = vehicle,
        min_ttc=min_ttc,
        get_asphalt_friction_coefficient = lambda : get_asphalt_friction_coefficient(),
        action_listener = lambda : stop_simulation()
    )
    return adas

## Gestione world

In [9]:
def load_map(world, map_name):
    if not world.get_map().name == f"Carla/Maps/{map_name}":
        world = client.load_world(map_name)

In [10]:
def destroy_all_vehicles(world):
    all_actors = world.get_actors()
    
    vehicles = all_actors.filter('vehicle.*')
    
    print(f"Numero di veicoli trovati: {len(vehicles)}")
    
    for vehicle in vehicles:
        vehicle.destroy()
        print(f"Veicolo distrutto: {vehicle.id}")

In [11]:
def destroy_all_pedestrians(world):
    all_actors = world.get_actors()
    
    pedestrians = all_actors.filter('walker.pedestrian.*')
    
    print(f"Numero di pedoni trovati: {len(pedestrians)}")
    
    for pedestrian in pedestrians:
        pedestrian.destroy()
        print(f"Pedone distrutto: {pedestrian.id}")

In [12]:
def disable_traffic_lights():
    traffic_lights = world.get_actors().filter('traffic.traffic_light')
    for traffic_light in traffic_lights:
        traffic_light.set_state(carla.TrafficLightState.Green)  # Imposta su verde
        traffic_light.set_green_time(99999)  # Mantiene il verde per un tempo molto lungo
        traffic_light.freeze(True)  # Disabilita la logica automatica

In [13]:
def enable_traffic_lights():

    traffic_lights = world.get_actors().filter('traffic.traffic_light')
    for traffic_light in traffic_lights:
        traffic_light.freeze(False)  # Riattiva il controllo automatico
        traffic_light.reset_group()  # Resetta lo stato e il ciclo del semaforo
        print(f"Semaforo {traffic_light.id} ripristinato al comportamento predefinito.")

## Creazione entità

In [14]:
def spawn_vehicle(world, location, index, y = 0):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('vehicle.*')[index]

    print(vehicle_bp)
    spawn_point = carla.Transform()
    spawn_point.location = location
    spawn_point.rotation = carla.Rotation(yaw = y)
    vehicle = world.try_spawn_actor(vehicle_bp, spawn_point)
    return vehicle

In [15]:
def actor_vehicle(world, vehicle, desitination, speed = 60):
    world.wait_for_tick()
    actor_agent = BasicAgent(vehicle)
    actor_agent.set_destination(desitination)
    actor_agent.set_target_speed(speed)
    actor_agent.ignore_vehicles()
    world.wait_for_tick()
    return actor_agent

In [16]:
def spawn_walker(start_location, yaw = 0):
    walker_bp = random.choice(world.get_blueprint_library().filter("walker.pedestrian.*"))
    walker_bp.set_attribute('is_invincible', 'false')

    trans = carla.Transform()
    trans.location = start_location
    trans.rotation = carla.Rotation(yaw = yaw)

    return world.spawn_actor(walker_bp, trans)

## Gestione movimenti

In [17]:
def move_forward(vehicle, throttle):
    control = carla.VehicleControl()
    control.throttle = throttle
    vehicle.apply_control(control)

In [18]:
def move_agent(vehicle, actor_agent):
    vehicle.apply_control(actor_agent.run_step())

In [19]:
def move_forward_agent(vehicle, actor_agent,x,y):
    forward_location = vehicle.get_location() + carla.Location(x=x, y=y, z=0)  # Adjust 'x', 'y' based on vehicle's orientation
    actor_agent.set_destination(forward_location)
    vehicle.apply_control(actor_agent.run_step())

In [20]:
def stop_vehicle(vehicle):
    control = carla.VehicleControl()
    control.throttle = 0.0
    control.brake = 1.0
    control.hand_brake = True
    vehicle.apply_control(control)

In [21]:
def move_pedestrian(x=1,y =0,z = 0):
    walker_control = carla.WalkerControl()
    walker_control.speed = 1.5  # Velocità in m/s (positivo in avanti)
    walker_control.direction = carla.Vector3D(x, y, z)
    return walker_control

In [22]:
def get_vehicle_velocity(vehicle):
    velocity_vector = vehicle.get_velocity()
    velocity = (velocity_vector.x**2 + velocity_vector.y**2 + velocity_vector.z**2)**0.5
    if vehicle.get_control().reverse:
        velocity = -velocity
    return velocity

## Camera

In [23]:
def print_spectator_position():
    location = world.get_spectator().get_transform().location
    rotation = world.get_spectator().get_transform().rotation
    msg = f"Il veicolo è stato posizionato in: x={location.x}, y={location.y}, z={location.z}"
    print(msg)
    print(location)
    print(rotation)

In [24]:
def set_spectator_location(location, rotation):
    spectator = world.get_spectator()
    spectator.set_transform(carla.Transform(location, rotation))

In [25]:
def run_camera(camera, vehicle):
    return threading.Thread(target=camera, args=(vehicle,))

In [26]:
def view_from_above(vehicle):
    while run:
        distance_above = 50.0  # Altezza della telecamera sopra il veicolo
        spectator = world.get_spectator()
        
        vehicle_location = vehicle.get_location()
        
        
        spectator_location = carla.Location(
                x=vehicle_location.x,  # Stessa posizione X del veicolo
                y=vehicle_location.y,  # Stessa posizione Y del veicolo
                z=vehicle_location.z + distance_above  # Posizionata più in alto rispetto al veicolo
            )

        spectator_rotation = carla.Rotation(
            pitch=-90.0,  # Guardare direttamente verso il basso
            yaw=0,  # Allineato con la rotazione del veicolo
            roll=0.0
        )
        
        spectator.set_transform(carla.Transform(spectator_location, spectator_rotation))
    

In [27]:
def view_from_behind(vehicle):
    while run:
        distance_behind = 14.0  # How far behind the car the spectator will be
        height = 5.0  # Height of the camera (how high above the car it will be)
        distance_to_the_side = 0
        
        spectator = world.get_spectator()
        
        vehicle_location = vehicle.get_location()
        vehicle_rotation = vehicle.get_transform().rotation
        
        spectator_location = carla.Location(
                x=vehicle_location.x - distance_behind * vehicle_rotation.get_forward_vector().x + distance_to_the_side * vehicle_rotation.get_right_vector().x,
                y=vehicle_location.y - distance_behind * vehicle_rotation.get_forward_vector().y + distance_to_the_side * vehicle_rotation.get_right_vector().y,
                z=vehicle_location.z + height  # Keeping the camera above the vehicle
            )

            # Set the spectator's transform to follow the vehicle
    
        spectator.set_transform(carla.Transform(spectator_location, vehicle_rotation))
    

# Scenari

## Scenario 0: Viaggio senza ostacoli


Verificare che in condiziondi normali il veicolo viaggi tranquillamente

### SETUP

In [28]:
load_map(world, "Town10HD_Opt")

In [29]:
disable_traffic_lights()

In [30]:
# percorso da seguire
wps = [
    carla.Location(x=-41.5, y=117, z=1),
    carla.Location(x=-41.5, y=45, z=1),
    carla.Location(x=30, y=28.3, z=1)
]

### run

In [31]:
destroy_all_vehicles(world)

vehicle = spawn_vehicle(world, wps[0], 0, -90)
world.wait_for_tick() # necessario per creare l'agente

actor_agent = actor_vehicle(world, vehicle, wps[1], 50)

wp = 1


try:
    run = True

    run_sync_simulation()
    
    adas = attach_adas(world, vehicle, 0.3)
    run_camera(view_from_behind, vehicle).start()
    
    while run:
        if actor_agent.done():
            print("The target has been reached, searching for another target")
            wp = (wp +1) % len(wps)
            actor_agent.set_destination(wps[wp])
            
        vehicle.apply_control(actor_agent.run_step())
        
    stop_vehicle(vehicle)
    
    time.sleep(2)
except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    run = False
finally:
    print("done")
    adas.destroy()


Numero di veicoli trovati: 0
ActorBlueprint(id=vehicle.audi.a2,tags=[vehicle, audi, a2])
run
The target has been reached, searching for another target
Send `WARNING` to topic `carla/fcw_state`
The target has been reached, searching for another target
The target has been reached, searching for another target
The target has been reached, searching for another target
The target has been reached, searching for another target
simulation stopped
done
stop


## Scenario 1: Veicolo fermo o parcheggiato sulla carreggiata


Presenza di un veicolo parcheggiato direttamente davanti lungo la corsia di marcia.

In [32]:
load_map(world, "Town10HD_Opt")

In [33]:
disable_traffic_lights()

In [72]:
destroy_all_vehicles(world)

vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=117, z=1), 2, -90)
destination = carla.Location(x=-41.5, y=45, z=1)
actor_agent = actor_vehicle(world, vehicle, destination, 120)
# osbstacle_vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=100, z=1), 1, -90)
# osbstacle_vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=70, z=1), 1, -90)
osbstacle_vehicle = spawn_vehicle(world, destination, 1, -90)

try:
    run = True
    
    run_sync_simulation()
    adas = attach_adas(world, vehicle, 0.1)

    run_camera(view_from_above, vehicle).start()

    while run:
        vehicle.apply_control(actor_agent.run_step())
        move_forward(vehicle, 0.7)
    stop_vehicle(vehicle)
    
    time.sleep(2)
except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    run = False

finally:
    print("done")
    adas.destroy()



Numero di veicoli trovati: 2
Veicolo distrutto: 134
Veicolo distrutto: 135
ActorBlueprint(id=vehicle.audi.tt,tags=[vehicle, audi, tt])
ActorBlueprint(id=vehicle.nissan.micra,tags=[vehicle, nissan, micra])
run
Send `WARNING` to topic `carla/fcw_state`
Send `ACTION` to topic `carla/fcw_state`
stop
stop
done


## Senario 2: Veicolo davanti in movimento con una velocità maggiore

In [ ]:
load_map(world, "Town10HD_Opt")

In [ ]:
disable_traffic_lights()

In [44]:
# percorso da seguire
wps = [
    carla.Location(x=-41.5, y=117, z=1),
    carla.Location(x=-41.5, y=45, z=1),
    carla.Location(x=30, y=28.3, z=1)
]

In [45]:
destroy_all_vehicles(world)

Numero di veicoli trovati: 3
Veicolo distrutto: 62
Veicolo distrutto: 61
Veicolo distrutto: 60


In [46]:
destroy_all_vehicles(world)

vehicle = spawn_vehicle(world, wps[0], 1, -90)
obstacle_vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=100, z=1), 1, -90)

actor_agent = actor_vehicle(world, vehicle, destination, 50)
obstacle_actor_agent = actor_vehicle(world, obstacle_vehicle, destination, 60)

destination = carla.Location(x=-41.5, y=45, z=1)

wp_a = 1
wp_b = 1

try:
    run = True
    adas = attach_adas(world, vehicle, 0.4)
    run_sync_simulation()    
    run_camera(view_from_above, vehicle).start()
    while run:
        if actor_agent.done():
            wp_a = (wp_a +1) % len(wps)
            actor_agent.set_destination(wps[wp_a])
        if obstacle_actor_agent.done():
            wp_b = (wp_b +1) % len(wps)
            obstacle_actor_agent.set_destination(wps[wp_b])

        vehicle.apply_control(actor_agent.run_step())
        obstacle_vehicle.apply_control(obstacle_actor_agent.run_step())
    
except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    run = False
finally:
    print("done")
    adas.destroy()


Numero di veicoli trovati: 0
ActorBlueprint(id=vehicle.nissan.micra,tags=[vehicle, nissan, micra])
ActorBlueprint(id=vehicle.nissan.micra,tags=[vehicle, nissan, micra])
run
Send `WARNING` to topic `carla/fcw_state`
simulation stopped
done
stop


## Senario 3: Veicolo davanti in movimento con una velocità minore

In [ ]:
load_map(world, "Town10HD_Opt")

In [ ]:
disable_traffic_lights()

In [47]:
# percorso da seguire
wps = [
    carla.Location(x=-41.5, y=117, z=1),
    carla.Location(x=-41.5, y=45, z=1),
    carla.Location(x=30, y=28.3, z=1)
]

In [51]:
destroy_all_vehicles(world)

vehicle = spawn_vehicle(world, wps[0], 1, -90)
obstacle_vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=100, z=1), 1, -90)

actor_agent = actor_vehicle(world, vehicle, destination, 60)
obstacle_actor_agent = actor_vehicle(world, obstacle_vehicle, destination, 30)

destination = carla.Location(x=-41.5, y=45, z=1)

wp_a = 1
wp_b = 1

try:
    run = True
    adas = attach_adas(world, vehicle, 0.4)
    run_sync_simulation()
    run_camera(view_from_above, vehicle).start()

    while run:
        if actor_agent.done():
            wp_a = (wp_a +1) % len(wps)
            actor_agent.set_destination(wps[wp_a])
        if obstacle_actor_agent.done():
            wp_b = (wp_b +1) % len(wps)
            obstacle_actor_agent.set_destination(wps[wp_b])

        vehicle.apply_control(actor_agent.run_step())
        obstacle_vehicle.apply_control(obstacle_actor_agent.run_step())

    stop_vehicle(vehicle)
    stop_vehicle(obstacle_vehicle)
    
except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    run = False
finally:
    print("done")
    adas.destroy()


Numero di veicoli trovati: 2
Veicolo distrutto: 73
Veicolo distrutto: 74
ActorBlueprint(id=vehicle.nissan.micra,tags=[vehicle, nissan, micra])
ActorBlueprint(id=vehicle.nissan.micra,tags=[vehicle, nissan, micra])
run
Send `WARNING` to topic `carla/fcw_state`
Send `ACTION` to topic `carla/fcw_state`
stop
done
stop


## Scenario 4 : Pedone che attraversa le strisce pedonali


Presenza di un pedone che attraversa le strisce pedonali mentre il veicolo di riferimento è a pochi metri da esse

### SETUP

In [ ]:
load_map(world, "Town10HD_Opt")

In [28]:
disable_traffic_lights()

### SIMULAZIONE SEMPLICE

In [31]:
destroy_all_vehicles(world)
destroy_all_pedestrians(world)

vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=80, z=1), 1, -90)
world.wait_for_tick()

walker = spawn_walker(carla.Location(x=-41.5, y=37, z=1))
destination = carla.Location(x=-41.5, y=10, z=1)

actor_agent = actor_vehicle(world, vehicle, destination, 120)

try:
    run = True
    
    adas = attach_adas(world, vehicle, 0.4)
    run_sync_simulation()
    
    run_camera(view_from_behind, vehicle).start()

    while run:
        vehicle.apply_control(actor_agent.run_step())
            
    stop_vehicle(vehicle)
    
except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    run = False
finally:
    print("done")
    adas.destroy()



Numero di veicoli trovati: 0
Numero di pedoni trovati: 1
Pedone distrutto: 271
ActorBlueprint(id=vehicle.nissan.micra,tags=[vehicle, nissan, micra])
run
Send `WARNING` to topic `carla/fcw_state`
Send `ACTION` to topic `carla/fcw_state`
stop
done
stop


### Pedone in movimento

In [ ]:
destroy_all_pedestrians(world)
walker = spawn_walker(carla.Location(x=-35, y=36.6, z=1), 180)

In [36]:
destroy_all_vehicles(world)
destroy_all_pedestrians(world)

vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=80, z=1), 1, -90)
world.wait_for_tick()

# walker = spawn_walker(carla.Location(x=-51, y=36.6, z=1))
walker = spawn_walker(carla.Location(x=-35, y=36.6, z=1), 180)
controller_w = move_pedestrian(-1,0,0)

destination = carla.Location(x=-41.5, y=72, z=1)

actor_agent = actor_vehicle(world, vehicle, destination, 35)

move = False

try:
    run = True
    
    adas = attach_adas(world, vehicle, 0.4)
    run_sync_simulation()
    run_camera(view_from_behind, vehicle).start()

    while run:
        if actor_agent.done():
            move = True
            actor_agent.set_destination(carla.Location(x=-41.5, y=10, z=1))
        if move:
            walker.apply_control(controller_w)
        vehicle.apply_control(actor_agent.run_step())
        
    stop_vehicle(vehicle)
#     vehicle.destroy()

except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    run = False
finally:
    print("done")
    adas.destroy()



Numero di veicoli trovati: 1
Veicolo distrutto: 285
Numero di pedoni trovati: 1
Pedone distrutto: 286
ActorBlueprint(id=vehicle.nissan.micra,tags=[vehicle, nissan, micra])
run
Send `WARNING` to topic `carla/fcw_state`
Send `ACTION` to topic `carla/fcw_state`
stop
done


stop


In [ ]:
destroy_all_pedestrians(world)
walker = spawn_walker(carla.Location(x=-44, y=36.6, z=1))

In [ ]:
destroy_all_vehicles(world)
vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=50, z=1), 0, -90)


In [ ]:
print_spectator_position()

In [ ]:
destroy_all_vehicles(world)
destroy_all_pedestrians(world)

In [ ]:
vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=70, z=1), 0, -90)
world.wait_for_tick()

## Salita

In [37]:
load_map(world, "Town03")

RuntimeError: time-out of 10000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000

In [ ]:
destroy_all_vehicles(world)

In [ ]:
vehicle = spawn_vehicle(world, carla.Location(x=178.759705, y=-5.8, z=1), 2, 180)
osbstacle_vehicle = spawn_vehicle(world, carla.Location(x=152.5, y=-33.249748, z=4), 0, -90) 
stop_vehicle(osbstacle_vehicle)

In [46]:
destroy_all_vehicles(world)

vehicle = spawn_vehicle(world, carla.Location(x=178.759705, y=-5.8, z=1), 2, 180)
world.wait_for_tick()

destination = carla.Location(x=152.5, y=-33.249748, z=4)

actor_agent = actor_vehicle(world, vehicle, destination, 120)

#Close
obstacle_vehicle = spawn_vehicle(world, carla.Location(x=153.095108, y=-58.077873, z=7), 0, -90) 
   
# obstacle_vehicle = spawn_vehicle(world, destination, 0, -90) 
stop_vehicle(obstacle_vehicle)

try:
    run = True
    adas = attach_adas(world, vehicle, min_ttc=0.4)
    run_sync_simulation()
    
    while run:
        vehicle.apply_control(actor_agent.run_step())

    stop_vehicle(vehicle)

finally:
    print("done")
    adas.destroy()


Numero di veicoli trovati: 2
Veicolo distrutto: 393
Veicolo distrutto: 392
ActorBlueprint(id=vehicle.audi.tt,tags=[vehicle, audi, tt])
ActorBlueprint(id=vehicle.audi.a2,tags=[vehicle, audi, a2])
run
Send `ACTION` to topic `carla/fcw_state`
stop
done


stop
